# SF Park Meter Model

In [ ]:
import pandas as pd
import numpy as np
import dask
import dask.dataframe as dd

In [ ]:
path = r'C:\Users\nelms\Documents\Penn\MUSA-508\MUSA508_FINAL_SFPARK\meter_starts_ends.parquet'
#path = r'C:\Users\nelms\Documents\Penn\MUSA-508\MUSA508_FINAL_SFPARK\meter_starts_ends.parquet'
hours_actions_df = dd.read_parquet(path).set_index('trans_id').persist()

# #############

def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

#hours_actions_df = hours_actions_df.repartition(npartitions=10)

#hours_actions_df = hours_actions_df.compute()
#################

In [ ]:
def round_down(value, decimals):
    factor = 1 / (10 ** decimals)
    return (value // factor) * factor
diff = .00001
bins = [0, 9, 12, 15, 18 + diff, 24, 1000]
labels = [
    '12a to {}a'.format(bins[1]),           # 12 - 9
    '{}a to {}p'.format(bins[1], round(bins[2])),       # 9 - 12
    '{}p to {}p'.format(round(bins[2]), bins[3]-12),    # 12 - 3
    '{}p to {}p'.format(bins[3]-12, round(bins[4])-12), # 3 - 6
    '{}p to 12a'.format(round(bins[4])-12),  # 6 - 12
    'ERROR'
    ]
good_bins = labels[1:-2]

#############

my_list = np.array([7,9.3,11.5,18.3,21,8,12.1,15.333,9.11])
indexes = np.digitize(my_list,bins) - 1
[labels[i] for i in indexes]

def f(x):
    # return math.sqrt(x)
    return np.sqrt(x)

bin_i = lambda t: labels[t]
vfunc = np.vectorize(bin_i)

In [ ]:
counter = 0

def grouper(row, low_cut_off_time=9, high_cut_off_time=18, diff = .00001):
    hours_actions = row[['time','action']].values.tolist()

    global counter
    counter = counter + 1
    if float(counter/10000).is_integer():
        print(counter)

    while hours_actions[0][1] == 'interval':
        hours_actions.pop(0)

    while hours_actions[-1][1] == 'interval':
        hours_actions.pop(-1)

    past_action = ''
    keep = []
    for hour, action in hours_actions:
        if (action != 'interval')and(action==past_action):
            if action == 'start':
                pass
            elif action == 'end':
                keep.pop()
                past_hour = hour
                past_action = action
                keep.append([hour, action])
        elif action!=past_action:
            # if action == 'end':
            #     if (hour == 18)or(hour == 18.0):
            #         hour = 17.99999
            #     elif (hour == 12)or(hour == 12.0):
            #         hour = 11.99999
            #     elif (hour == 15)or(hour == 15.0):
            #         hour = 14.99999
            past_action = action
            keep.append([hour, action])
    hours_actions = keep
    #start_count = len([act for hr,act in keep if act=='start'])

    past_action = ''
    keep = []
    for hour, action in hours_actions:
        if (action == 'interval'):
            if(past_action == 'start'):
                correct_end = hour-diff
                # if (hour == 12)or(hour == 12.0):
                #     correct_end = 11.99999
                keep.append([correct_end,'end'])
                keep.append([hour,'start'])
                past_action = 'start'
            else:
                pass
        else:
            past_action = action
            keep.append([hour, action])
            
    hours_actions = keep
    past_action = ''
    keep = []
    for hour, action in hours_actions:
        if (action=='end')and((hour == 18)or(hour == 18.0)):
            hour = 17.99999
        if (hour >= low_cut_off_time) and (hour < high_cut_off_time):
            if (action==past_action):
                if action == 'start':
                    pass
                elif action == 'end':
                    keep.pop()
                    past_hour = hour
                    past_action = action
                    keep.append([hour, action])
            elif action!=past_action:
                past_hour = hour
                past_action = action
                keep.append([hour, action])
        else:
            pass
    #start_count = len([act for hr,act in keep if act=='start'])

    if (len(keep) % 2) != 0:
        try:
            if keep[0][1] == 'end':
                keep.pop(0)

            if keep[-1][1] == 'start':
                keep.pop(-1)
        except:
            pass
    try:
        test = np.array(keep)[:,1]
        keep = np.array(keep)[:,0].astype("float")
        try:
            keep = np.reshape(keep, (round(len(keep)/2), 2))
        except:
            print(round(len(keep)/2, 2))
            print(test)
            print(keep)
        try:
            time_diff = keep[:,1] - keep[:,0]
        except:
            time_diff = []
            for row in keep:
                try:
                    time_diff.append(row[1] - row[0])
                except:
                    print(row[1], row[0])
                    print(type(row[1]))
                    time_diff.append(0)  

        try:
            indexes = np.digitize(keep[:,0], bins) - 1
            indexes = vfunc(indexes)
        except: 
            indexes = []
            for num in keep[:,0]:
                idx = np.digitize(num, bins) - 1
                idx = idx[0]
                try:
                    indexs.append(bins[idx])
                except:
                    print(idx, ' ', num)
                    indexs.append(999)
            indexes = np.array(indexes)
    except:
        print('prob')
        time_diff = [999]
        indexes = ['Error']
    # hours_actions_df = pd.DataFrame(keep, columns=['time','action'])
    #return pd.DataFrame(keep, columns=['time','action'])
    #return pd.Series((keep[:,0], my_list, bins_num), index=['time','action', 'bins'])
    return pd.Series((time_diff, indexes), index=['diff','bins'])

#trans_day_actions = hours_actions_df.map_partitions(grouper, meta={'time': 'int64', 'action': 'str'})
trans_day_actions = hours_actions_df.groupby('trans_id')[['time','action']].apply(grouper, meta={'diff':float,'bins':str}) #, meta=pd.DataFrame(dtype=, columns=['time','action'])
print('done grouping')
trans_day_actions = trans_day_actions.compute()
print('group')
path = r'C:\Users\nelms\Documents\Penn\MUSA-508\MUSA508_FINAL_SFPARK\meter_counts_trans_all.parquet'
trans_day_actions.to_parquet(path)


In [ ]:
import pandas as pd
import numpy as np
import dask
import dask.dataframe as dd

path = r"C:\Users\nelms\Documents\Penn\MUSA-508\MUSA508_FINAL_SFPARK\meter_counts_trans_all.parquet"
trans_day_actions = dd.read_parquet(path).reset_index().repartition(npartitions=10)
trans_day_actions = trans_day_actions.values.compute_chunk_sizes()
trans_day_actions = trans_day_actions.persist()

In [ ]:
def round_down(value, decimals):
    factor = 1 / (10 ** decimals)
    return (value // factor) * factor
diff = .00001
bins = [0, 9, 12, 15, 18 + diff, 24, 1000]
labels = [
    '12a to {}a'.format(bins[1]),           # 12 - 9
    '{}a to {}p'.format(bins[1], round(bins[2])),       # 9 - 12
    '{}p to {}p'.format(round(bins[2]), bins[3]-12),    # 12 - 3
    '{}p to {}p'.format(bins[3]-12, round(bins[4])-12), # 3 - 6
    '{}p to 12a'.format(round(bins[4])-12),  # 6 - 12
    'ERROR'
    ]
good_bins = labels[1:-2]

new_cols = ['meter_day_bin', 'meter_id', 'day', 'bin', 'time', 'poss']

counter = 0

def add_missing_bins(row):
    bins = row[2]
    diffs = row[1]
    diffs = [float(i) for i in diffs]
    trans_ids = row[0]

    good_dict = {k:0 for k in good_bins}

    global counter

    if counter == 0:
        print(diffs)

    counter = counter + 1
    if float(counter/100000).is_integer():
        print(counter)

    for time,binn in zip(diffs,bins):
        if binn in good_bins:
            good_dict[binn]+=time

    meter_id, day = trans_ids.split('_')
    day = int(day)
    all_good = np.array([[trans_ids + '_' + binn, meter_id, day, binn, time, '3'] for binn,time in good_dict.items()])
    return all_good
#add_missing_bins = np.vectorize(add_missing_bins)

def get_new_array(arr):
    return np.vstack(np.apply_along_axis(add_missing_bins, 1, arr))

meters_day_bin = dask.array.map_blocks(get_new_array, trans_day_actions, meta = np.array((), dtype=str))

meters_day_bin = meters_day_bin.compute()
print('done processing')

times = meters_day_bin[:,4].astype(float)
times[times > 3] = 3
meters_day_bin[:,4] = times
meters_day_bin = meters_day_bin[meters_day_bin[:, 0].argsort()]
print('done cleaning')

print('Columns: ', new_cols)
meters = np.unique(meters_day_bin[:,1])
days = np.unique(meters_day_bin[:,2].astype(int))
bins = np.unique(meters_day_bin[:,3])
print('Meters', len(meters), 'Days', len(days), 'Bins', len(bins))
print('Possible Combination Size:', len(meters) * len(days) * len(bins))
print('Actual Size', meters_day_bin.shape)

path = r'C:\Users\nelms\Documents\Penn\MUSA-508\MUSA508_FINAL_SFPARK\meters_day_bin_count.parquet'
pd.DataFrame(meters_day_bin, columns=new_cols).to_parquet(path)


In [3]:
path = r'C:\Users\nelms\Documents\Penn\MUSA-508\MUSA508_FINAL_SFPARK\Meter_Regular_Operating_Good.csv'
regs = pd.read_csv(path)[['block_id','meter_id']].drop_duplicates().set_index('meter_id')['block_id'].to_dict()

import geopandas as gpd
path = r"C:\Users\nelms\Documents\Penn\MUSA-508\MUSA508_FINAL_SFPARK\data\Parking Meters.geojson"
pm = gpd.read_file(path)
pm = pm[~pm['blockface_id'].isna()][['post_id', 'blockface_id', 'pm_district_id', 'geometry']]
pm['blockface_id'] = pm['blockface_id'].astype(float).astype(int).astype(str)
pm['pm_district_id'] = pm['pm_district_id'].astype(float).astype(int).astype(str)
pm_block_dict = pm.set_index('post_id')['blockface_id'].to_dict()
pm_dist_dict = pm.set_index('post_id')['pm_district_id'].to_dict()

pm_block_dict = {k:v for k,v in pm_block_dict.items() if k in regs.keys()}
pm_dist_dict = {k:v for k,v in pm_dist_dict.items() if k in regs.keys()}

path = r"C:\Users\nelms\Documents\Penn\MUSA-508\MUSA508_FINAL_SFPARK\data\Blockfaces.geojson"
bf = gpd.read_file(path)
bf = bf[~bf['sfpark_id'].isna()][['sfpark_id', 'geometry']].rename(columns={'sfpark_id':'blockface_id'})
bf['blockface_id'] = bf['blockface_id'].astype(int).astype(str)

In [4]:
import pandas as pd
import numpy as np
import dask
import dask.dataframe as dd

path = r'C:\Users\nelms\Documents\Penn\MUSA-508\MUSA508_FINAL_SFPARK\meters_day_bin_count.parquet'
meters_day_bin = dd.read_parquet(path)
meters_day_bin = meters_day_bin.values.compute_chunk_sizes().compute()

In [5]:
new_cols = ['meter_day_bin', 'meter_id', 'day', 'bin', 'time', 'poss']
final_cols = new_cols

block_counts = pd.DataFrame(meters_day_bin, columns=final_cols)
block_counts = block_counts[
    (block_counts['meter_id'].isin(pm_block_dict.keys()))&(block_counts['meter_id'].isin(pm_dist_dict.keys()))
    ]
block_counts['block_id'] = block_counts['meter_id'].map(pm_block_dict)
block_counts['dist_id'] = block_counts['meter_id'].map(pm_dist_dict)

block_counts['time'] = block_counts['time'].astype(float)
block_counts['poss'] = block_counts['poss'].astype(int)
block_counts['day'] = block_counts['day'].astype(int).astype(str).str.zfill(3)
block_counts['block_day_bin'] = block_counts['block_id'] + '_' + block_counts['day'] + '_' + block_counts['bin']

In [6]:
def round_down(value, decimals):
    factor = 1 / (10 ** decimals)
    return (value // factor) * factor
diff = .00001
bins = [0, 9, 12, 15, 18 + diff, 24, 1000]
labels = [
    '12a to {}a'.format(bins[1]),           # 12 - 9
    '{}a to {}p'.format(bins[1], round(bins[2])),       # 9 - 12
    '{}p to {}p'.format(round(bins[2]), bins[3]-12),    # 12 - 3
    '{}p to {}p'.format(bins[3]-12, round(bins[4])-12), # 3 - 6
    '{}p to 12a'.format(round(bins[4])-12),  # 6 - 12
    'ERROR'
    ]
good_bins = labels[1:-2]
good_bins_hours = {binn:str(((e+1)*3)+6).zfill(2) for e,binn in enumerate(good_bins)}
good_bins_hours

{'9a to 12p': '09', '12p to 3p': '12', '3p to 6p': '15'}

In [7]:
block_group = block_counts.drop_duplicates().groupby('block_day_bin').agg(
    block_id = ('block_id', 'first'),
    dist_id = ('block_id', 'first'),
    meter_ids = ('meter_id',list),
    meter_count = ('meter_id', 'count'),
    day = ('day', 'first'),
    bin = ('bin','first'),
    occ_hours = ('time','sum'),
    tot_hours = ('poss','sum')
).sort_values(['block_id','day'])

block_group['occ_perc'] = block_group['occ_hours'] / block_group['tot_hours']
block_group['meter_ids'] = block_group['meter_ids'].apply(lambda l: ', '.join(set(l)))

block_group['day of week'] = block_group['day'] + ' 2019'
block_group['date'] = pd.to_datetime(block_group['day of week'], format="%j %Y")
block_group['day of week'] = block_group['date'].dt.day_name()
print('clean')
block_group['datetime'] = block_group['date'].astype(str) + ' ' + block_group['bin'].map(good_bins_hours)
block_group['datetime'] = pd.to_datetime(block_group['datetime'], format = '%Y-%m-%d %H')
print('datetime')
path = r'C:\Users\nelms\Documents\Penn\MUSA-508\MUSA508_FINAL_SFPARK\data\block_count_occ.parquet'
block_group.to_parquet(path)

block_group.head(20)

clean
datetime


,block_id,dist_id,meter_ids,meter_count,day,bin,occ_hours,tot_hours,occ_perc,day of week,date,datetime
block_day_bin,,,,,,,,,,,,
102021_002_12p to 3p,102021,102021,102-02990,1,002,12p to 3p,2.333333,3,0.777778,Wednesday,2019-01-02,2019-01-02 12:00:00
102021_002_3p to 6p,102021,102021,102-02990,1,002,3p to 6p,0.000000,3,0.000000,Wednesday,2019-01-02,2019-01-02 15:00:00
102021_002_9a to 12p,102021,102021,102-02990,1,002,9a to 12p,1.433323,3,0.477774,Wednesday,2019-01-02,2019-01-02 09:00:00
102021_003_12p to 3p,102021,102021,102-02990,1,003,12p to 3p,0.000000,3,0.000000,Thursday,2019-01-03,2019-01-03 12:00:00
102021_003_3p to 6p,102021,102021,102-02990,1,003,3p to 6p,1.549990,3,0.516663,Thursday,2019-01-03,2019-01-03 15:00:00
102021_003_9a to 12p,102021,102021,102-02990,1,003,9a to 12p,1.300000,3,0.433333,Thursday,2019-01-03,2019-01-03 09:00:00
102021_004_12p to 3p,102021,102021,102-02990,1,004,12p to 3p,0.833333,3,0.277778,Friday,2019-01-04,2019-01-04 12:00:00
102021_004_3p to 6p,102021,102021,102-02990,1,004,3p to 6p,1.466657,3,0.488886,Friday,2019-01-04,2019-01-04 15:00:00
102021_004_9a to 12p,102021,102021,102-02990,1,004,9a to 12p,0.333323,3,0.111108,Friday,2019-01-04,2019-01-04 09:00:00
